In [40]:
import sys
sys.path.append('..')
from db import *
import re
import numpy as np
import dateparser

In [41]:
pd.set_option('display.max_columns', 100)

## Load the RT data

In [42]:
rt_data = query('SELECT * FROM rt_data')

In [43]:
rt_data.head()

,url,title,critic_rating,audience_rating,network,premiere_date,genre,executive_producers
0,https://www.rottentomatoes.com/tv/live_pd,Live PD,None,75,A&E Network,None,Special Interest,"['Dan Cesareo', 'David Doss', 'George McTeague..."
1,https://www.rottentomatoes.com/tv/black_jesus,Black Jesus,None,70,Cartoon Network,None,None,"['Aaron McGruder', 'Mike Clattenburg', 'Norman..."
2,https://www.rottentomatoes.com/tv/love_after_l...,Love After Lockup,None,None,WE tv,"Jan 12, 2018",Documentary,"['Matt Sharp', 'Dan Adler', 'Lauren P. Gellert..."
3,https://www.rottentomatoes.com/tv/z_nation,Z Nation,None,79,SyFy,"Sep 12, 2014",Action & Adventure,"['Karl Schaefer', 'David Michael Latt', 'John ..."
4,https://www.rottentomatoes.com/tv/timeline,Timeline,None,None,Hulu,None,None,None


## Create the Streaming Table

### Key conversion table

In [44]:
streaming_urls = query('SELECT * FROM amazon_netflix_rt_urls')

In [45]:
streaming_urls.rename({'by_the_numbers_name' : 'title'}, inplace = True, axis = 'columns')

In [46]:
streaming_urls.head()

,title,rt_name,url
0,Bosch,bosch,https://www.rottentomatoes.com/tv/bosch
1,Hand of God,hand_of_god,https://www.rottentomatoes.com/tv/hand_of_god
2,The Man in the High Castle,the_man_in_the_high_castle,https://www.rottentomatoes.com/tv/the_man_in_t...
3,Mad Dogs,mad_dogs,https://www.rottentomatoes.com/tv/mad_dogs
4,Goliath,goliath,https://www.rottentomatoes.com/tv/goliath


### List of Streaming Shows

In [47]:
streaming_shows = query('SELECT * FROM netflix_amazon_shows')

In [48]:
streaming_shows.head()

,title,genre,premiere_date,network
0,Bosch,Detective fiction,13-Feb-15,Amazon
1,Hand of God,Psychological thriller,4-Sep-15,Amazon
2,The Man in the High Castle,Alternative history,20-Nov-15,Amazon
3,Mad Dogs,Drama,22-Jan-16,Amazon
4,Goliath,Legal drama,14-Oct-16,Amazon


### Perform the Merge

In [49]:
temp_df      = rt_data.merge(streaming_urls,  on = 'url',                                      suffixes = ('_rt', '_streaming'), validate = '1:1')
streaming_df = temp_df.merge(streaming_shows, left_on = 'title_streaming', right_on = 'title', suffixes = ('_rt', '_streaming'), validate = '1:1')

In [50]:
streaming_df.head()

,url,title_rt,critic_rating,audience_rating,network_rt,premiere_date_rt,genre_rt,executive_producers,title_streaming,rt_name,title,genre_streaming,premiere_date_streaming,network_streaming
0,https://www.rottentomatoes.com/tv/bloodline,Bloodline,62,76,Netflix,"Mar 20, 2015",Drama,"['Todd A. Kessler', 'Daniel Zelman', 'Glenn Ke...",Bloodline,bloodline,Bloodline,Thriller,20-Mar-15,Netflix
1,https://www.rottentomatoes.com/tv/the_ranch,The Ranch,63,73,Netflix,"Apr 1, 2016",Comedy,"['Don Reo', 'Jim Patterson']",The Ranch,the_ranch,The Ranch,Sitcom,1-Apr-16,Netflix
2,https://www.rottentomatoes.com/tv/good_girls_r...,Good Girls Revolt,73,97,Amazon,"Nov 4, 2015",Drama,"['Dana Calvo', 'Lynda Obst', 'Darlene Hunt', '...",Good Girls Revolt,good_girls_revolt,Good Girls Revolt,Historical period drama,28-Oct-16,Amazon
3,https://www.rottentomatoes.com/tv/the_mind_exp...,"The Mind, Explained",None,100,Netflix,"Sep 12, 2019",Documentary,None,"The Mind, Explained",the_mind_explained,"The Mind, Explained",Docu-series,12-Sep-19,Netflix
4,https://www.rottentomatoes.com/tv/bonding,Bonding,64,78,Netflix,"Apr 24, 2019",Comedy,"['Rightor Doyle', 'Jacob Perlin', 'Nina Sorian...",Bonding,bonding,Bonding,Dark comedy,24-Apr-19,Netflix


In [51]:
streaming_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 264 entries, 0 to 263
Data columns (total 14 columns):
url                        264 non-null object
title_rt                   264 non-null object
critic_rating              166 non-null object
audience_rating            239 non-null object
network_rt                 264 non-null object
premiere_date_rt           254 non-null object
genre_rt                   249 non-null object
executive_producers        161 non-null object
title_streaming            264 non-null object
rt_name                    264 non-null object
title                      264 non-null object
genre_streaming            264 non-null object
premiere_date_streaming    264 non-null object
network_streaming          264 non-null object
dtypes: object(14)
memory usage: 30.9+ KB


In [53]:
streaming_df.rename({'premiere_date_streaming' : 'premiere_date',
                     'genre_rt' : 'genre',
                     'network_streaming' : 'network',
                     'rt_name'           : 'rt_id'
                    }, inplace = True, axis = 'columns')

In [54]:
streaming_df['premiere_date'] = pd.to_datetime(streaming_df.premiere_date)

In [55]:
streaming_df['critic_rating'] = streaming_df.critic_rating.astype(float)
streaming_df['audience_rating'] = streaming_df.audience_rating.astype(float)

In [59]:
streaming_df_clean = streaming_df[['rt_id', 'title', 'critic_rating', 'audience_rating', 'network',  'genre', 'executive_producers', 'premiere_date']]

In [60]:
streaming_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 264 entries, 0 to 263
Data columns (total 8 columns):
rt_id                  264 non-null object
title                  264 non-null object
critic_rating          166 non-null float64
audience_rating        239 non-null float64
network                264 non-null object
genre                  249 non-null object
executive_producers    161 non-null object
premiere_date          264 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(5)
memory usage: 18.6+ KB


In [61]:
# Change if_exists to 'replace' to actually make changes
#  leave as 'fail' to make this safe to re-run
streaming_df_clean.to_sql('streaming', db, if_exists = 'replace', index = False)

## Create the Nielsen Table

### Load Key Conversion Table

In [19]:
nielsen_urls = query('SELECT * FROM rt_urls')

In [20]:
nielsen_urls.rename({'rt_name' : 'rt_id'}, inplace = True, axis = 'columns')

In [21]:
nielsen_urls.head()

,by_the_numbers_name,rt_id,url
0,LIVE PD,live_pd,https://www.rottentomatoes.com/tv/live_pd
1,BLACK JESUS,black_jesus,https://www.rottentomatoes.com/tv/black_jesus
2,LOVE AFTER LOCKUP,love_after_lockup,https://www.rottentomatoes.com/tv/love_after_l...
3,REAL TIME WITH BILL MAHER,real_time_with_bill_maher,https://www.rottentomatoes.com/tv/real_time_wi...
4,MOM,mom,https://www.rottentomatoes.com/tv/mom


### Load the Nielsen Data

In [22]:
nielsen_data = query('SELECT * FROM tv_by_the_numbers_articles')

In [23]:
nielsen_data.head()

,time,show,demo_rating,viewers,tags,article_title,url,notes,network,table_id
0,10:40 PM,NBA REGULAR SEASON: UTAH/LA LAKERS,0.7,"1,783","['post-538457', 'post', 'type-post', 'status-p...",The strength of the NBA continued at the top o...,https://tvbythenumbers.zap2it.com/daily-rating...,['000s'],ESPN,12
1,8:12 PM,NBA REGULAR SEASON: DALLAS/NEW ORLEANS,0.4,"1,103","['post-538457', 'post', 'type-post', 'status-p...",The strength of the NBA continued at the top o...,https://tvbythenumbers.zap2it.com/daily-rating...,['000s'],ESPN,13
2,9:00 PM,GOLD RUSH,0.4,"2,076","['post-538457', 'post', 'type-post', 'status-p...",The strength of the NBA continued at the top o...,https://tvbythenumbers.zap2it.com/daily-rating...,['000s'],DISCOVERY,14
3,9:00 PM,LIVE PD,0.4,"1,534","['post-538457', 'post', 'type-post', 'status-p...",The strength of the NBA continued at the top o...,https://tvbythenumbers.zap2it.com/daily-rating...,['000s'],A&E,15
4,1:13 AM,SPORTSCENTER 1AM,0.4,845,"['post-538457', 'post', 'type-post', 'status-p...",The strength of the NBA continued at the top o...,https://tvbythenumbers.zap2it.com/daily-rating...,['000s'],ESPN,16


### Perform the Merge

In [24]:
temp_df = rt_data.merge(nielsen_urls, on = 'url', validate = '1:m')
nielsen_df = temp_df.merge(nielsen_data, left_on = 'by_the_numbers_name', right_on = 'show', suffixes = ('_rt', '_tvbtn'), validate = '1:m')

### Clean the Nielsen Data and colled the appropriate columns

In [25]:
def clean_demo_rating(r):
    try:
        if '/' in r:
            return float(r[:r.find('/')])
        else:
            return float(r)
    except:
        return np.nan

In [26]:
nielsen_df.demo_rating = nielsen_df.demo_rating.map(clean_demo_rating)

In [27]:
nielsen_df.columns

Index(['url_rt', 'title', 'critic_rating', 'audience_rating', 'network_rt',
       'premiere_date', 'genre', 'executive_producers', 'by_the_numbers_name',
       'rt_id', 'time', 'show', 'demo_rating', 'viewers', 'tags',
       'article_title', 'url_tvbtn', 'notes', 'network_tvbtn', 'table_id'],
      dtype='object')

In [28]:
months = ['Jan', 'January', 'Feb', 'February', 'Mar', 'March', 'Apr', 'April', 'May', 'Jun', 'June', 'Jul', 'July',
          'Aug', 'August', 'Sep', 'Sept', 'September', 'Oct', 'October', 'Nov', 'November', 'Dec', 'December']
month_re  = '(' + '|'.join(months) + ')'
date_re   = '([0-3]?[0-9])'
year_re   = '(2?0?1[0-9])'
total_re  = '[-/_]'.join([month_re, date_re, year_re])
total_re

'(Jan|January|Feb|February|Mar|March|Apr|April|May|Jun|June|Jul|July|Aug|August|Sep|Sept|September|Oct|October|Nov|November|Dec|December)[-/_]([0-3]?[0-9])[-/_](2?0?1[0-9])'

In [29]:
def parse_date(url):
    look = re.search(total_re, url, re.IGNORECASE)
    if not look:
        return None
    
    findings = list(look.groups())
    if findings[2] == '215':
        findings[2] = '2015'
    out_date = dateparser.parse(' '.join(findings))
    if not out_date:
        print(f'Warning, failed to parse the date for {url}')
        return None
    
    return out_date.date()

In [30]:
# Warning: this is slow. Takes ~5 minutes.
dates = nielsen_df.url_tvbtn.map(parse_date)

In [31]:
nielsen_df['date'] = pd.to_datetime(dates)

In [32]:
nielsen_df.rename({'network_tvbtn' : 'network'}, inplace = True, axis = 'columns')

In [33]:
nielsen_df['critic_rating'] = nielsen_df.critic_rating.astype(float)
nielsen_df['audience_rating'] = nielsen_df.audience_rating.astype(float)
nielsen_df['demo_rating']     = nielsen_df.demo_rating.astype(float)

In [34]:
# Some data is duplicated so try to keep the data with 'final' in the url
#  Other duplicates appear to be: same tv show aired at different time slots and/or on different networks throughout the day

def is_final(s):
    m = re.search('final-ratings', s, re.IGNORECASE)
    if m:
        return True
    else:
        return False

nielsen_df['duplicated'] = nielsen_df.duplicated(subset = ['rt_id', 'date', 'time'], keep = False)

nielsen_df['is_final']   = nielsen_df.url_tvbtn.map(is_final)

nielsen_df['has_date']   = nielsen_df.date.isna().map(lambda b : not b)

nielsen_df['to_keep'] = ~(~nielsen_df.is_final & nielsen_df['duplicated'] & nielsen_df.has_date)

nielsen_df = nielsen_df[nielsen_df['to_keep']]

In [35]:
nielsen_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24795 entries, 0 to 25691
Data columns (total 25 columns):
url_rt                 24795 non-null object
title                  24795 non-null object
critic_rating          8071 non-null float64
audience_rating        19693 non-null float64
network_rt             24795 non-null object
premiere_date          9648 non-null object
genre                  22578 non-null object
executive_producers    16735 non-null object
by_the_numbers_name    24795 non-null object
rt_id                  24795 non-null object
time                   24795 non-null object
show                   24795 non-null object
demo_rating            24366 non-null float64
viewers                24795 non-null object
tags                   24795 non-null object
article_title          24795 non-null object
url_tvbtn              24795 non-null object
notes                  24795 non-null object
network                24791 non-null object
table_id               24795 non-nu

In [36]:
# Some data was poorly formatted and parsed incorrectly. Drop these rows
nielsen_df.drop(index = nielsen_df[nielsen_df.demo_rating > 10].index, inplace = True)

In [37]:
nielsen_df_clean = nielsen_df[['rt_id', 'title', 'critic_rating', 'audience_rating', 'network', 'genre', 'executive_producers', 'demo_rating', 'date']]

In [38]:
nielsen_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24792 entries, 0 to 25691
Data columns (total 9 columns):
rt_id                  24792 non-null object
title                  24792 non-null object
critic_rating          8071 non-null float64
audience_rating        19691 non-null float64
network                24788 non-null object
genre                  22575 non-null object
executive_producers    16734 non-null object
demo_rating            24363 non-null float64
date                   23149 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 1.9+ MB


In [39]:
# Change if_exists to 'replace' to actually make changes
#  leave as 'fail' to make this safe to re-run
# nielsen_df_clean.to_sql('nielsen', db, if_exists = 'fail', index = False)
nielsen_df_clean.to_sql('nielsen', db, if_exists = 'replace', index = False)